In general, there are three types of constraints:
- Constraint repair
- Penalty methods
- Feasibility-preserving operations

From an implementation standpoint, constraint repair takes the updated particles at time $t$ and applies some function that modifies the particle positions to be enforce feasibility. Penalty methods instead modify the computed objective score after the particle position update as a function of the new positions and the constraints. Finally, feasibility-preserving operations overload the algebraic operations (addition and multiplication) to guarantee feasibility (e.g., using the Aitchison geometry to enforce sum-to-one constraints). 

In [2]:
include("./src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, PSO, Objectives
using LinearAlgebra

In [3]:
function unit_hypercube_constraints(d::Int)
    I_d = Matrix{Float64}(I, d, d)
    A = vcat(I_d, -I_d)
    b = vcat(ones(d), zeros(d))
    return A, b
end

unit_hypercube_constraints (generic function with 1 method)

In [4]:
A, b = unit_hypercube_constraints(4)
experiment = Experiments.create(7, 4)
experiment = Experiments.with_linear_constraints(experiment, A, b)

Main.MetaDoE.Experiments.Experiment(Dict("4" => 4, "1" => 1, "2" => 2, "3" => 3), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; … ; -0.0 -0.0 -1.0 -0.0; -0.0 -0.0 -0.0 -1.0], [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]), 7, 4)

## Optimization Context
- Create initializer
- Get an enforcer
- Define the enforcer-specific parameters
- Define the optimization algorithm-specific parameters
- Run the optimization

In [5]:
initializer = Experiments.get_initializer(experiment)

(::Main.MetaDoE.Designs.var"#sample_constraints#9"{Random.TaskLocalRNG, Int64, Int64, Matrix{Float64}, Vector{Float64}, Int64, Polyhedra.DefaultLibrary{Float64}}) (generic function with 1 method)

## Optimization Test Case

In [6]:
constraints = ConstraintEnforcement.LinearConstraints(A, b)
linear_enforcer = ConstraintEnforcement.LinearEnforcer(constraints)
objective = PSO.create_objective(Objectives.griewank, linear_enforcer)

Main.MetaDoE.PSO.Objective(Main.MetaDoE.ConstraintEnforcement.var"#3#6"{Main.MetaDoE.ConstraintEnforcement.LinearConstraints}(Main.MetaDoE.ConstraintEnforcement.LinearConstraints([1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; … ; -0.0 -0.0 -1.0 -0.0; -0.0 -0.0 -0.0 -1.0], [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0])), Main.MetaDoE.PSO.var"#10#12"{typeof(Main.MetaDoE.Objectives.griewank)}(Main.MetaDoE.Objectives.griewank))

In [9]:
initializer = Experiments.get_initializer(experiment)
swarm = PSO.initialize_swarm(initializer, objective, PSO.default_hyperparams())
context = PSO.create_context(swarm, PSO.aggregate_results(save_world=true))
runner_state, history = PSO.optimize(context)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
8 rows, 5 cols, 16 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-8); columns 0(-5); elements 0(-16) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.0000000000e-01
HiGHS run time      :       

(Main.MetaDoE.PSO.RunnerState(Main.MetaDoE.PSO.Swarm(Main.MetaDoE.PSO.ParticleState([-6.601383853452078e-10 3.9807233273092013e-10 … -5.846429330245141e-9 -2.0238955703192155e-10; 1.6646315850320766e-10 -2.3114581768207983e-10 … -4.819743287198308e-9 -4.234814381042416e-9; … ; -2.9822040404835753e-10 1.7012146187590637e-9 … -2.476208017800708e-9 9.94384487184783e-10; 1.5943725082822345e-9 -5.765310931928068e-9 … 5.5904798605870514e-9 -1.5972883312364594e-10;;; 2.460251406542308e-9 -4.767337604312796e-9 … -3.084779759886022e-8 8.606096713906645e-9; 1.9021046090453314e-9 -6.041561170172421e-9 … -9.799895296862327e-9 2.6686359316925805e-8; … ; 1.0331276798104496e-9 -3.2942368417344455e-9 … -9.25915783760152e-9 -4.213203084470253e-9; -3.5187308839029315e-9 8.71477096098945e-10 … -2.566228835214031e-8 1.1015434336666237e-9;;; -9.33275036540591e-9 7.639031025805927e-9 … -3.317721503927198e-9 3.915508266434969e-9; 2.3690214483254487e-9 4.1571175746318925e-10 … -6.754315255368968e-9 -1.0116481

In [13]:
optimizer, optimal_score = PSO.get_optimizer(runner_state)
optimal_score

0.0